In [1]:
# packages
import pandas as pd
import numpy as np
from sklearn.model_selection  import train_test_split, GridSearchCV
from sklearn.feature_extraction import FeatureHasher
from sklearn import svm
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import mean_squared_error,accuracy_score, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor;

In [2]:
df = pd.read_csv('/dbfs/FileStore/tables/data_sonuc.csv')

In [3]:
df.isnull().sum()

Out[ 3 ]: season 0
home_team_api_id 0
away_team_api_id 0
sonuc 0
B365H 0
B365D 0
B365A 0
BWH 0
BWD 0
BWA 0
IWH 0
IWD 0
IWA 0
LBH 0
LBD 0
LBA 0
WHH 0
WHD 0
WHA 0
SJH 0
SJD 0
SJA 0
VCH 0
VCD 0
VCA 0
GBH 0
GBD 0
GBA 0
VAR_H 0
VAR_D 0
VAR_A 0
dtype: int64

In [4]:
corr_matrix = df.corr()
corr_matrix['sonuc'].sort_values(ascending = False)

Out[ 4 ]: sonuc 1.000000
B365H 0.204532
IWH 0.203769
BWH 0.203364
SJH 0.202731
LBH 0.202532
WHH 0.202295
GBH 0.199724
VCH 0.199638
VAR_H 0.088931
home_team_api_id 0.018475
GBD 0.004459
B365D 0.003328
LBD 0.003315
BWD 0.002738
VCD 0.002593
SJD 0.002146
WHD 0.001165
IWD -0.001102
VAR_D -0.001917
away_team_api_id -0.012111
VAR_A -0.017084
VCA -0.095633
WHA -0.101343
SJA -0.101688
GBA -0.102162
LBA -0.103883
B365A -0.104039
BWA -0.108233
IWA -0.111112
Name: sonuc, dtype: float64

In [5]:
x = df[["B365H","B365D","B365A","BWH","BWD","BWA","IWH","IWD","IWA","LBH","LBD","LBA","WHH","WHD","WHA","SJH","SJD","SJA","VCH","VCD","VCA","GBH","GBD","GBA"
]].copy()
y = df['sonuc'].copy()
#print (y)
#print(x.head())
# split and shuffle
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 0)

In [6]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators =1000, random_state=0)


In [7]:
rfc.fit(x_train,y_train)
print(rfc.feature_importances_)

[ 0.03843895 0.0304392 0.03816336 0.04509214 0.03964724 0.04782592
 0.03493194 0.03004308 0.04031897 0.03864161 0.03120272 0.03624431
 0.0431361 0.03389618 0.04055032 0.05399767 0.04569298 0.05354818
 0.04735495 0.03628826 0.04234203 0.05314875 0.04477116 0.05428399]

In [8]:
training_prediction = rfc.predict(x_train)
training_accuracy = accuracy_score(y_train, training_prediction) 

In [9]:
print (training_accuracy)

1.0

In [10]:
pred_tree = rfc.predict(x_test)

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
accuracy = accuracy_score(y_test,pred_tree)
precision = precision_score(y_test,pred_tree, average="micro")
recall = recall_score(y_test,pred_tree, average="micro")
f1 = f1_score(y_test,pred_tree, average="micro")
#confusion_matrix_ = confusion_matrix(test_y_j.values.ravel(),predicted_values)
print("accuracy: {}\nprecision: {}\nrecall: {}\nf1: {}".format(accuracy,precision,recall,f1)) 

accuracy: 0.5244299674267101
precision: 0.5244299674267101
recall: 0.5244299674267101
f1: 0.5244299674267101

In [12]:

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 200)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 30)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_search = RandomizedSearchCV(rfc, param_distributions=random_grid, cv=5, iid=False)

In [13]:
random_search.fit(x_train,y_train)

Out[ 17 ]: RandomizedSearchCV(cv=5, error_score='raise',
 estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
 max_depth=None, max_features='auto', max_leaf_nodes=None,
 min_impurity_split=1e-07, min_samples_leaf=1,
 min_samples_split=2, min_weight_fraction_leaf=0.0,
 n_estimators=1000, n_jobs=1, oob_score=False, random_state=0,
 verbose=0, warm_start=False),
 fit_params={}, iid=False, n_iter=10, n_jobs=1,
 param_distributions={'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 209, 218, 227, 236, 245, 254, 263, 272, 281, 290, 299, 308, 317, 326, 335, 344, 353, 362, 371, 380, 389, 398, 408, 417, 426, 435, 444, 453, 462, 471, 480, 489,...8, 61, 65, 68, 72, 75, 78, 82, 85, 89, 92, 96, 99, 103, 106, 110, None], 'bootstrap': [True, False]},
 pre_dispatch='2*n_jobs', random_state=None, refit=True,
 return_train_score=True, scoring=None, verbose=0)

In [14]:
predictions = random_search.predict(x_test)

In [15]:
accuracy = accuracy_score(y_test,predictions)
precision = precision_score(y_test,predictions, average="micro")
recall = recall_score(y_test,predictions, average="micro")
f1 = f1_score(y_test,predictions, average="micro")
#confusion_matrix_ = confusion_matrix(test_y_j.values.ravel(),predicted_values)
print("accuracy: {}\nprecision: {}\nrecall: {}\nf1: {}".format(accuracy,precision,recall,f1)) 

accuracy: 0.5333135919455138
precision: 0.5333135919455138
recall: 0.5333135919455138
f1: 0.5333135919455138